In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import numpy as np
import matplotlib.pyplot as plt
import feature_manager as fma
import classifier.multi_dnn_classifier as dnn
from random import randint
from keras import callbacks, losses
import visualizer
from keras.utils import np_utils
from sklearn.metrics import classification_report, confusion_matrix
import importlib
import tr_utils
import datetime

2023-01-29 14:37:37.120722: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-29 14:37:37.120890: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-01-29 14:37:37.120903: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-01-29 14:37:38.747999: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-01-29 14:37:38.748059: I tensorflow/compiler/xla/stream_exe

In [2]:
symbol = "BTCUSDT"
trade_tf = "1h"
granular_tf = "1m"

fm = fma.FeatureManager(
    target_col="trade_signal"
)

fm.import_trading_data(
    symbol=symbol,
    trade_timeframe=trade_tf,
)

fm.import_granular_data(
    symbol=symbol,
    granular_timeframe=granular_tf
)

Imported ../data/BTCUSDT-1h.csv with 46982 rows
Imported ../nocommit/BTCUSDT-1m.csv with 2817999 rows


In [3]:
fm.df["swing"] = (fm.df["High"]-fm.df["Low"])/fm.df["Close"]

In [4]:
fm.df["swing"].describe()

count    46982.000000
mean         0.011602
std          0.011129
min          0.000000
25%          0.005332
50%          0.008498
75%          0.013971
max          0.308296
Name: swing, dtype: float64

In [ ]:
importlib.reload(fma)

In [6]:
tp = 0.03
sl = 0.03
md = 12

fm.prepare_trade_forward_data(
    take_profit_rate=tp,
    stop_loss_rate=sl,
    max_duration=md,
)

Scanning 12 future timeframes to build trade signal: 
1  2  3  4  5  6  7  8  9  10  11  12  
Label producing completed. 
 Value counts:
0    28558
2     9560
1     8864
Name: trade_signal, dtype: int64


In [7]:
fm.build_features(lags=12)

Calculating technical features...

Adding features with lags 12: returns, dir, hashrate, fed_rate, gold, nasdaq, sp500, google_trend, sma, boll, boll7, boll14, boll21, min, min7, min14, min21, max, max7, max14, max21, mom, mom7, mom14, mom21, vol, vol7, vol14, vol21, obv, mfi7, mfi14, mfi21, rsi7, rsi14, rsi21, adx7, adx14, adx21, roc, roc7, roc14, roc21, atr7, atr14, atr21, bop, ad, adosc, trange, ado, willr7, willr14, willr21, dx7, dx14, dx21, trix, ultosc, high, low, 

Normalizing features with MaxAbs: returns_lag_1, returns_lag_2, returns_lag_3, returns_lag_4, returns_lag_5, returns_lag_6, returns_lag_7, returns_lag_8, returns_lag_9, returns_lag_10, returns_lag_11, returns_lag_12, dir_lag_1, dir_lag_2, dir_lag_3, dir_lag_4, dir_lag_5, dir_lag_6, dir_lag_7, dir_lag_8, dir_lag_9, dir_lag_10, dir_lag_11, dir_lag_12, hashrate_lag_1, hashrate_lag_2, hashrate_lag_3, hashrate_lag_4, hashrate_lag_5, hashrate_lag_6, hashrate_lag_7, hashrate_lag_8, hashrate_lag_9, hashrate_lag_10, hashrate_l

In [57]:
importlib.reload(dnn)

<module 'classifier.multi_dnn_classifier' from '/home/andy/CryptoTradingPlatform/TraderRobot/project/classifier/multi_dnn_classifier.py'>

In [58]:
DEFAULT_PRINT_METRICS = [
    "loss",
    "accuracy",
    "precision",
    "recall",
    "precision-0.65",
    "recall-0.65",
    "precision-0.80",
    "recall-0.80",
    "precision-0.95",
    "recall-0.95"
]

results_list = []
results = dnn.evaluate_classifier_k_folds(
    hu = 500,
    fm = fm,
    fold_number = 10,
    batch_size = 1024,
    set_class_weight = False,
    save_check_point = True,
    early_stop = True,
    rebalance = None,
    split_type="time_series_split",
    metrics=DEFAULT_PRINT_METRICS,
    set_initial_bias = True,
    dropout = True,
    dropout_rate = 0.3,
    shuffle_when_train = True,
    gpu = False,
    write_to_file = True
)
results_list.append(results)


DATA PREPARATION PARAMS:
trade_timeframe:1h
take_profit_rate:0.03
stop_loss_rate:0.03
max_duration:12
lags:12
fold_number:10
train_size:0.7
val_size:0.15
categorical_label:True
rebalance:None
split_type:time_series_split



DATA:
Total rows: 46821
Label 0: 28524(60.92%)
Label 1: 8820(18.84%)
Label 2: 9477(20.24%)
Splitting the data...

>>>>>> FOLD 1


DATA IN FOLD
Train: 3901, Validation: 3901, Test: 3901

Train:
Label 0: 1000(25.63%)
Label 1: 1497(38.37%)
Label 2: 1404(35.99%)

Validation:
Label 0: 1937(49.65%)
Label 1: 893(22.89%)
Label 2: 1071(27.45%)

Test:
Label 0: 2694(69.06%)
Label 1: 503(12.89%)
Label 2: 704(18.05%)

CLASSIFIER PARAMS:
hu:500
output_bias:[-0.2475961228030686, 0.15586698263442275, 0.09172918280055087]
loss:categorical_crossentropy
dropout:True
dropout_rate:0.3
learning_rate:0.0001
gpu:False
set_class_weight:False
save_check_point:True
early_stopping:True
patience:5
epochs:200
shuffle_when_train:True
batch_size:1024
class_weight:None


Epoch 1/200
4/4 [=========

/home/andy/anaconda3new/envs/tensor/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/andy/anaconda3new/envs/tensor/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/andy/anaconda3new/envs/tensor/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

In [ ]:
fm.df["gold_lag_1"].value_counts()

In [ ]:
metric_list = [
    "loss",
    "accuracy",
    "precision",
    "recall",
    "precision-0.65",
    "recall-0.65",
    "precision-0.80",
    "recall-0.80",
    "precision-0.95",
    "recall-0.95"
]

In [ ]:
results_list = []
results = dnn.evaluate_classifier(
    hu=100,
    fm=fm,
    laps=1,
    batch_size=20,
    set_class_weight=False,
    save_check_point=False,
    early_stopping=True,
    shuffle_before_split=False,
    dropout=True,
    dropout_rate=0.3,
    gpu=False,
    metrics=metric_list,
    write_to_file=True
)
results_list.append(results)

In [ ]:
results_df = pd.DataFrame(results)

In [ ]:
results_df.to_csv("../out/evaluate.csv")

In [ ]:
importlib.reload(dnn)

In [ ]:
classifier = dnn.MultiDNNClassifer()

filename = "../logs/report/{}.txt".format(datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
report_file = open(filename,'w')

dataset = classifier.prepare_data(
    data = fm.df,
    cols = fm.cols,
    shuffle_before_split= False,
    categorical_label=True,
    rebalance="over",
    target_col="trade_signal",
    file = report_file
)

In [ ]:
initial_bias = tr_utils.init_imbalanced_bias(
    y_train=dataset[1]
)

classifier.configure(
    hu = 100, 
    dropout=True, 
    dropout_rate = 0.3,
    input_dim=len(fm.cols),
    output_bias=initial_bias,
    class_num=3
)

test_result = classifier.run(
    gpu = False,
    dataset = dataset,
    epochs = 500,
    shuffle_when_train = False,
    patience = 5,
    early_stop = True,
    save_check_point = True,
    set_class_weight = False,
    batch_size = 128,
    file = report_file
)

var_df = pd.DataFrame(data=classifier.viann_callback.varScores, index = fm.cols)
var_df.columns = ["var"]
var_df.sort_values(by=["var"],ascending=False)
